<a href="https://colab.research.google.com/github/matibag1714977/LabiagiProject/blob/main/Progetto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Progetto-Scene Classification**

Import

In [14]:
import numpy as np      #to perform mathematical operations on arrays and matrices
import matplotlib.pyplot as plt  #to visualize images and plot graphs

#provides algoithms and tools for data analysis and modeling(classification,regression,clustering)
from sklearn.model_selection import train_test_split        
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

#It provides a user-friendly interface for building and training deep learning models
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

# It provides various tools and functions for image and video processing, including image filtering, feature detection, and object tracking.
import cv2
import imghdr
import os
import tensorflow as tf


from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential, Model


import torch
import torchvision
import torchvision.transforms as transforms

import splitfolders



Import dataset Prova

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

folder_id = '1pFM_HpOjh0qbAadk0-1Fz89mwLHrnQHB?usp=share_link'
data_path = '/content/gdrive/MyDrive/Dataset/input_data'


# mount the folder
drive.mount('/content/gdrive', force_remount=True)




Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


Per dividere la cartella training,testing validation

In [5]:
!pip install split_folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
input_folder ="/content/gdrive/MyDrive/Dataset/input_data"
output="/content/gdrive/MyDrive/Dataset/processed_data"
splitfolders.ratio(input_folder,output,seed=42,ratio=(.6, .2, .2))

Copying files: 6660 files [01:32, 72.16 files/s]


In [23]:
img_height, img_width = (224,224)
batch_size=32

train_data_dir=f"/content/gdrive/MyDrive/Dataset/processed_data/train"
valid_data_dir=f"/content/gdrive/MyDrive/Dataset/processed_data/val"
test_data_dir=f"/content/gdrive/MyDrive/Dataset/processed_data/test"

In [27]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode="categorical")

valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical")

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode="categorical")



Found 3991 images belonging to 10 classes.
Found 1327 images belonging to 10 classes.
Found 1342 images belonging to 10 classes.


In [28]:
EPOCHS = 100

base_model = ResNet50(include_top=False, weights="imagenet")

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)

predictions = Dense(train_generator.num_classes, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["acc"])




history = model.fit(train_generator,
                    validation_data=valid_generator,
                    epochs=EPOCHS)

94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/100
125/125 [==============================] - 1052s 8s/step - loss: 0.4201 - acc: 0.8825 - val_loss: 0.1718 - val_acc: 0.9457
Epoch 2/100
 34/125 [=======>......................] - ETA: 9:13 - loss: 0.1746 - acc: 0.9467

KeyboardInterrupt: ignored

Mostra alcune immagini

In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image



# Target image size
image_size = (224, 224)

# Path to the main SUN397 dataset directory
dataset_path = "/content/gdrive/MyDrive/Dataset/Prova"

# List of the 10 classes to display
classes_to_display = ["beach", "temple", "forest", "mountain", "airport_terminal", "river", "street", "playground", "skyscraper","bridge"]

# List of all the image filepaths in the selected classes
selected_images = []
for class_name in classes_to_display:
    class_path = os.path.join(dataset_path, class_name)
    for filename in os.listdir(class_path):
        filepath = os.path.join(class_path, filename)
        if filepath.endswith(".jpg"):
            selected_images.append((filepath, class_name))  # Add class name to the tuple

# Sample 8 random images from the selected images
sample_images = random.sample(selected_images, 8)

# Display the images with their filenames as titles
fig, ax = plt.subplots(2, 4, figsize=(10, 5))
for i in range(2):
    for j in range(4):
        # Open the image using the PIL library
        img = Image.open(sample_images[i*4+j][0])
        # Resize the image to the target size
        img = img.resize(image_size)
        # Convert the image to RGB format (if it's grayscale)
        img = img.convert('RGB')
        # Display the image in the subplot
        ax[i][j].imshow(img)
        ax[i][j].axis('off')
        # Get the filename of the image
        filename = os.path.basename(sample_images[i*4+j][0])
        # Get the class name of the image
        class_name = sample_images[i*4+j][1]
        # Set the title of the subplot to the class name and filename
        ax[i][j].set_title(f"{class_name}")
plt.show()


Normalizing